In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, median_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from scipy.stats import *
import h3
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
rides = pd.read_csv("../rides_2022-09-14_17to19.csv")
rides.rename(columns={'id': 'ride_id'}, inplace=True)
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city
count,2.426000e+04,2.426000e+04,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,24260.000000,2.426000e+04,24260.0
mean,2.010495e+06,2.376807e+07,35.693980,51.401119,35.694668,51.397562,1003.226051,1236.469992,4.956898e+09,1.0
std,1.440427e+06,1.800928e+07,0.096191,0.143753,0.095899,0.152071,725.298062,1034.221594,3.405802e+05,0.0
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.636840,50.016300,24.000000,120.000000,4.956287e+09,1.0
25%,5.573940e+05,5.043194e+06,35.663319,51.331473,35.661003,51.330028,464.000000,504.000000,4.956594e+09,1.0
50%,1.992264e+06,2.400925e+07,35.723982,51.404408,35.723633,51.401817,799.000000,905.000000,4.956982e+09,1.0
75%,3.323714e+06,3.831664e+07,35.755246,51.465694,35.754699,51.470322,1347.000000,1650.000000,4.957167e+09,1.0
max,4.346548e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,7794.000000,9088.000000,4.957626e+09,1.0


In [3]:
khatkesh_columns = ['ride_id','driver_id','a_t_a_result.arrival_a_t_a','a_t_a_result.boarding_a_t_a','a_t_a_result.ride_a_t_a','a_t_a_result.arrival_probe_result.probe.point.lat','a_t_a_result.arrival_probe_result.probe.point.lon','a_t_a_result.arrival_probe_result.probe.timestamp','a_t_a_result.arrival_probe_result.confidence','a_t_a_result.arrival_probe_result.h3_index', 'a_t_a_result.arrival_probe_result.k_ring_level','a_t_a_result.boarding_probe_result.probe.point.lat','a_t_a_result.boarding_probe_result.probe.point.lon','a_t_a_result.boarding_probe_result.probe.timestamp','a_t_a_result.boarding_probe_result.confidence','a_t_a_result.boarding_probe_result.h3_index','a_t_a_result.boarding_probe_result.k_ring_level','a_t_a_result.final_destination_probe_result.probe.point.lat','a_t_a_result.final_destination_probe_result.probe.point.lon','a_t_a_result.final_destination_probe_result.probe.timestamp','a_t_a_result.final_destination_probe_result.confidence','a_t_a_result.final_destination_probe_result.h3_index','a_t_a_result.final_destination_probe_result.k_ring_level','a_t_a_result.destination_probe_result.probe.point.lat','a_t_a_result.destination_probe_result.probe.point.lon','a_t_a_result.destination_probe_result.probe.timestamp','a_t_a_result.destination_probe_result.confidence','a_t_a_result.destination_probe_result.h3_index','a_t_a_result.destination_probe_result.k_ring_level','a_t_a_result.extra_destination_probe_result.probe.point.lat','a_t_a_result.extra_destination_probe_result.probe.point.lon','a_t_a_result.extra_destination_probe_result.probe.timestamp','a_t_a_result.extra_destination_probe_result.confidence','a_t_a_result.extra_destination_probe_result.h3_index','a_t_a_result.extra_destination_probe_result.k_ring_level','pickup_a_d_d_result.distance','pickup_a_d_d_result.confidence','pickup_a_d_d_result.route_ratio','pickup_a_d_d_result.g_p_s_ratio','ride_a_d_d_result.distance','ride_a_d_d_result.confidence','ride_a_d_d_result.route_ratio','ride_a_d_d_result.g_p_s_ratio','total_a_d_d_confidence','in_ride_allotment','e_d_d','clickhouse_time','hash']

khatkesh = pd.read_csv("../khatkesh_2022-09-14_17to20.csv", names=khatkesh_columns)
khatkesh.describe()

,ride_id,driver_id,a_t_a_result.arrival_a_t_a,a_t_a_result.boarding_a_t_a,a_t_a_result.ride_a_t_a,a_t_a_result.arrival_probe_result.probe.point.lat,a_t_a_result.arrival_probe_result.probe.point.lon,a_t_a_result.arrival_probe_result.probe.timestamp,a_t_a_result.arrival_probe_result.confidence,a_t_a_result.arrival_probe_result.k_ring_level,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,6.098400e+05,6.098400e+05,609840.000000,6.098400e+05,6.098400e+05,609840.000000,609840.000000,6.098400e+05,609840.000000,609840.000000,...,609840.000000,609840.000000,609840.000000,609840.000000,609840.000000,609840.000000,609840.000000,609840.000000,609840.000000,6.098400e+05
mean,4.956710e+09,2.384851e+06,186.565196,1.932879e+16,6.883807e+07,34.591887,51.914119,2.359385e+15,0.732979,1.162698,...,0.000335,0.015180,13.615310,0.856565,0.000126,0.706963,0.867031,0.044672,8.404534,9.236962e+18
std,4.912347e+05,1.344603e+06,146.839880,5.968084e+17,2.094513e+09,2.550840,3.542795,2.086083e+17,0.342170,1.441072,...,0.016178,0.017419,17.575679,0.189884,0.005198,0.346683,0.179733,0.323489,12.480039,5.326899e+18
min,4.844714e+09,1.000000e+00,0.000000,0.000000e+00,1.000000e+00,25.277332,44.829102,1.662865e+09,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.465604e+13
25%,4.956423e+09,1.207348e+06,91.000000,8.000000e+00,4.080000e+02,32.688224,50.834740,1.663160e+09,0.591716,0.000000,...,0.000000,0.005618,4.604000,0.745633,0.000000,0.474879,0.788933,0.000000,2.787000,4.629244e+18
50%,4.956712e+09,2.445052e+06,170.000000,3.600000e+01,6.960000e+02,35.706343,51.409451,1.663162e+09,0.769231,1.000000,...,0.000000,0.011765,8.639000,0.958135,0.000000,0.886792,0.957863,0.000000,5.191000,9.238150e+18
75%,4.957015e+09,3.622817e+06,260.000000,1.240000e+02,1.193000e+03,36.262882,52.525059,1.663165e+09,1.000000,2.000000,...,0.000000,0.020833,16.389000,1.000000,0.000000,0.978723,1.000000,0.000000,9.871000,1.385267e+19
max,4.957470e+09,4.346548e+06,7954.000000,1.844674e+19,6.379876e+10,39.661167,61.168961,1.844674e+19,1.000000,4.000000,...,1.000000,1.000000,1888.735107,1.000000,1.000000,1.000000,1.000000,34.022999,1741.516968,1.844671e+19


In [4]:
rides_khatkesh = pd.merge(rides, khatkesh, on="ride_id")
rides_khatkesh.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,2.082500e+04,2.082500e+04,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,2.082500e+04,20825.0,...,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,20825.000000,2.082500e+04
mean,1.999775e+06,2.383175e+07,35.693529,51.401175,35.694669,51.398380,942.397935,1162.141945,4.956832e+09,1.0,...,0.000215,0.013655,14.471063,0.873934,0.000118,0.736780,0.882062,0.014971,8.667095,9.189144e+18
std,1.436861e+06,1.797496e+07,0.096443,0.144898,0.096133,0.151682,670.597667,974.159366,3.145274e+05,0.0,...,0.012298,0.014508,14.610381,0.175027,0.002828,0.325293,0.165155,0.146202,8.962879,5.342897e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.016300,24.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.209000,3.680090e+14
25%,5.483600e+05,5.111832e+06,35.662746,51.331078,35.660850,51.330456,447.000000,483.000000,4.956540e+09,1.0,...,0.000000,0.005076,4.884000,0.803479,0.000000,0.567164,0.827302,0.000000,2.923000,4.565318e+18
50%,1.975867e+06,2.410657e+07,35.723858,51.404060,35.723995,51.402237,762.000000,861.000000,4.956874e+09,1.0,...,0.000000,0.010000,9.970000,0.964556,0.000000,0.904639,0.963044,0.000000,5.987000,9.109705e+18
75%,3.310593e+06,3.833837e+07,35.754833,51.466557,35.754784,51.470547,1253.000000,1522.000000,4.957101e+09,1.0,...,0.000000,0.018182,19.007999,1.000000,0.000000,0.978947,1.000000,0.000000,11.234000,1.383284e+19
max,4.346548e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,6541.000000,8672.000000,4.957458e+09,1.0,...,1.000000,0.500000,220.848297,1.000000,0.204274,1.000000,1.000000,4.430000,143.080002,1.844592e+19


In [5]:
rides = rides_khatkesh[(rides_khatkesh['eta'] > 180) &
                       (rides_khatkesh['eta'] < 10800) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] > 180) &
                       (rides_khatkesh['a_t_a_result.ride_a_t_a'] < 10800)]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.995100e+04,1.995100e+04,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,1.995100e+04,19951.0,...,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,19951.000000,1.995100e+04
mean,1.998629e+06,2.367655e+07,35.694714,51.399828,35.695908,51.396894,975.000702,1202.144404,4.956830e+09,1.0,...,0.000120,0.012422,15.017412,0.875576,0.000096,0.738031,0.883830,0.015057,8.987522,9.172071e+18
std,1.437602e+06,1.796485e+07,0.095208,0.140696,0.094870,0.147962,664.787487,973.012366,3.139771e+05,0.0,...,0.007574,0.010151,14.682233,0.169591,0.002098,0.323891,0.160056,0.147448,9.010713,5.344776e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.016300,181.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.194000,0.015032,0.000000,0.000000,0.027182,0.000000,0.598000,3.680090e+14
25%,5.474410e+05,4.983128e+06,35.664698,51.331452,35.663388,51.330749,482.000000,525.000000,4.956539e+09,1.0,...,0.000000,0.005102,5.404000,0.805343,0.000000,0.571429,0.828412,0.000000,3.209500,4.540209e+18
50%,1.972606e+06,2.393345e+07,35.724102,51.403915,35.724280,51.401997,792.000000,897.000000,4.956872e+09,1.0,...,0.000000,0.009804,10.496239,0.963008,0.000000,0.904762,0.962616,0.000000,6.310000,9.085335e+18
75%,3.311235e+06,3.817876e+07,35.755157,51.465082,35.755127,51.469389,1280.000000,1561.000000,4.957101e+09,1.0,...,0.000000,0.017241,19.606633,1.000000,0.000000,0.977859,1.000000,0.000000,11.529000,1.382678e+19
max,4.346331e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,6541.000000,8672.000000,4.957442e+09,1.0,...,1.000000,0.052632,220.848297,1.000000,0.099045,1.000000,1.000000,4.430000,143.080002,1.844592e+19


In [6]:
rides = rides[rides['ride_a_d_d_result.confidence'] > 0.6]
rides.describe()

,accepted_driver_id,passenger_id,source_lat,source_lng,destination_lat,destination_lng,eta,ata,ride_id,city,...,pickup_a_d_d_result.route_ratio,pickup_a_d_d_result.g_p_s_ratio,ride_a_d_d_result.distance,ride_a_d_d_result.confidence,ride_a_d_d_result.route_ratio,ride_a_d_d_result.g_p_s_ratio,total_a_d_d_confidence,in_ride_allotment,e_d_d,hash
count,1.726400e+04,1.726400e+04,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,1.726400e+04,17264.0,...,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,17264.000000,1.726400e+04
mean,2.004751e+06,2.366595e+07,35.694460,51.399300,35.695705,51.396517,968.105827,1192.088740,4.956830e+09,1.0,...,0.000067,0.013237,15.755814,0.930941,0.000056,0.828822,0.935722,0.016814,8.906675,9.202475e+18
std,1.439244e+06,1.795832e+07,0.095280,0.141639,0.094679,0.148424,661.319188,965.774165,3.136741e+05,0.0,...,0.002537,0.010083,15.128015,0.100924,0.001438,0.224497,0.090770,0.156545,8.928459,5.359270e+18
min,1.010000e+02,1.375000e+03,35.249897,50.903470,34.641853,50.016300,181.000000,120.000000,4.956287e+09,1.0,...,0.000000,0.000000,0.194000,0.600579,0.000000,0.016854,0.518909,0.000000,0.598000,3.680090e+14
25%,5.545290e+05,5.014237e+06,35.664250,51.330625,35.663042,51.329991,479.000000,522.000000,4.956540e+09,1.0,...,0.000000,0.005814,5.767500,0.905261,0.000000,0.765652,0.903373,0.000000,3.197750,4.546019e+18
50%,1.975852e+06,2.389992e+07,35.724190,51.402439,35.724215,51.400720,784.000000,887.000000,4.956871e+09,1.0,...,0.000000,0.010526,11.204000,0.978166,0.000000,0.930403,0.976414,0.000000,6.251000,9.122385e+18
75%,3.321656e+06,3.820608e+07,35.755235,51.464877,35.754804,51.468814,1266.000000,1545.000000,4.957100e+09,1.0,...,0.000000,0.018182,20.626437,1.000000,0.000000,0.983607,1.000000,0.000000,11.436500,1.388810e+19
max,4.346206e+06,5.750996e+07,35.839420,52.794230,36.259760,52.792877,6541.000000,8672.000000,4.957432e+09,1.0,...,0.156830,0.052632,220.848297,1.000000,0.093853,1.000000,1.000000,4.430000,143.080002,1.844592e+19


In [7]:
rides = rides[['eta', 'ata', 'a_t_a_result.ride_a_t_a', 'source_lat', 'source_lng', 'destination_lat', 'destination_lng', 'e_d_d']]

In [8]:
rides['source_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.source_lat, row.source_lng, 4), axis=1)
rides['dest_h3_4'] = rides.apply(lambda row: h3.geo_to_h3(row.destination_lat, row.destination_lng, 4), axis=1)

In [10]:
# eta_edd_h3 = rides[["eta", "e_d_d", "source_h3_4", "dest_h3_4"]]
# ata = rides["a_t_a_result.ride_a_t_a"]

In [11]:
# eta_edd_h3_train, eta_edd_h3_test, ata_train, ata_test = train_test_split(eta_edd_h3, ata, test_size=0.05, random_state=73)
# print(len(eta_edd_h3_train))
# print(len(eta_edd_h3_test))

11813
622


In [12]:
# dummy_train_rides = pd.get_dummies(eta_edd_h3_train, prefix={'source_h3_4':'source_h3_4',
#                                                        'dest_h3_4':'dest_h3_4'},
#                           drop_first=True)
# dummy_train_rides.describe()

,eta,e_d_d,source_h3_4_842cd59ffffffff,source_h3_4_842cd5bffffffff,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cd59ffffffff,dest_h3_4_842cd5bffffffff,dest_h3_4_842cf07ffffffff,dest_h3_4_842cf27ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000,11813.000000
mean,896.309490,8.395520,0.003894,0.000423,0.843139,0.021163,0.054008,0.024972,0.051299,0.003555,0.000423,0.000254,0.000085,0.836367,0.021248,0.050876,0.024719,0.002032,0.059680
std,622.467211,8.574888,0.062283,0.020570,0.363685,0.143934,0.226044,0.156048,0.220617,0.059524,0.020570,0.015935,0.009201,0.369958,0.144215,0.219754,0.155273,0.045030,0.236903
min,181.000000,0.689000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,444.000000,2.956000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,714.000000,5.683000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1156.000000,10.777000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5231.000000,116.138000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# geo_noise = np.count_nonzero(dummy_train_rides, axis=0) < 10
#
# cols = dummy_train_rides.columns
# for col, noise in zip(cols, geo_noise):
#     if noise:
#         # DOESN'T REINDEX
#         eta_edd_h3_train = eta_edd_h3_train[dummy_train_rides[col] == 0]
#         ata_train = ata_train[dummy_train_rides[col] == 0]
#
# eta_edd_h3_train.describe()

/var/folders/yq/707ysz_j3jlggmtk5clx13rm0000gn/T/ipykernel_46750/592031391.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  eta_edd_h3_train = eta_edd_h3_train[dummy_train_rides[col] == 0]


,eta,e_d_d
count,11804.000000,11804.000000
mean,895.475008,8.373800
std,619.786057,8.472199
min,181.000000,0.689000
25%,444.000000,2.954750
50%,714.000000,5.683000
75%,1156.000000,10.772000
max,5231.000000,116.138000


In [14]:
# print(np.count_nonzero(dummy_train_rides, axis=0))
# print(dummy_train_rides.columns)

[11813 11813    46     5  9960   250   638   295   606    42     5     3
     1  9880   251   601   292    24   705]
Index(['eta', 'e_d_d', 'source_h3_4_842cd59ffffffff',
       'source_h3_4_842cd5bffffffff', 'source_h3_4_842cf31ffffffff',
       'source_h3_4_842cf33ffffffff', 'source_h3_4_842cf35ffffffff',
       'source_h3_4_842cf37ffffffff', 'source_h3_4_842cf3dffffffff',
       'dest_h3_4_842cd59ffffffff', 'dest_h3_4_842cd5bffffffff',
       'dest_h3_4_842cf07ffffffff', 'dest_h3_4_842cf27ffffffff',
       'dest_h3_4_842cf31ffffffff', 'dest_h3_4_842cf33ffffffff',
       'dest_h3_4_842cf35ffffffff', 'dest_h3_4_842cf37ffffffff',
       'dest_h3_4_842cf39ffffffff', 'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [15]:
# ohe = ColumnTransformer(
#     [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False),['source_h3_4', 'dest_h3_4'])],
#     remainder = 'passthrough'
#     ).fit(eta_edd_h3_train)

In [16]:
# eta_edd_h3_train_transformed = ohe.transform(eta_edd_h3_train)

In [17]:
# eta_edd_h3_test_transformed = ohe.transform(eta_edd_h3_test)

In [18]:
# reg = LinearRegression().fit(eta_edd_h3_train_transformed, ata_train)

In [19]:
# print(reg.coef_)
# print(reg.intercept_)

[ -25.75887343  224.94473332  -19.01559462   89.69035555 -118.84195984
  -79.42441842  -71.59424257  117.64382932 -155.64796364   39.22240546
 -111.21523618   62.08345633  121.00354353  -72.53872651   -0.55130831
    1.53506348  -12.22265237]
-161.57924243144953


In [20]:
# ata_train_pred = reg.predict(eta_edd_h3_train_transformed)
# ata_test_pred = reg.predict(eta_edd_h3_test_transformed)

In [21]:
# eta_edd_h3_train_transformed

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.79100000e+03, 1.66520004e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.63700000e+03, 1.57670002e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.76800000e+03, 2.09650002e+01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 7.09000000e+02, 5.75500011e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.09800000e+03, 5.74200010e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 4.83000000e+02, 3.08599997e+00]])

In [22]:
# print("ETA train MAE", mean_absolute_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train MAE", mean_absolute_error(ata_train, ata_train_pred))
#
# print("ETA test MAE ", mean_absolute_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test MAE", mean_absolute_error(ata_test, ata_test_pred))
#
#
# print("ETA train MSE", mean_squared_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train MSE", mean_squared_error(ata_train, ata_train_pred))
#
# print("ETA test MSE ", mean_squared_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test MSE", mean_squared_error(ata_test, ata_test_pred))
#
#
# print("ETA train explained_variance_score", explained_variance_score(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train explained_variance_score", explained_variance_score(ata_train, ata_train_pred))
#
# print("ETA test explained_variance_score ", explained_variance_score(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test explained_variance_score", explained_variance_score(ata_test, ata_test_pred))
#
#
# print("ETA train median_absolute_error", median_absolute_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train median_absolute_error", median_absolute_error(ata_train, ata_train_pred))
#
# print("ETA test median_absolute_error ", median_absolute_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test median_absolute_error", median_absolute_error(ata_test, ata_test_pred))
#
#
# print("ETA train r2_score", r2_score(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train r2_score", r2_score(ata_train, ata_train_pred))
#
# print("ETA test r2_score ", r2_score(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test r2_score", r2_score(ata_test, ata_test_pred))
#
#
# print("ETA train MAPE", mean_absolute_percentage_error(ata_train, eta_edd_h3_train_transformed[:, 15]))
# print("prediction train MAPE", mean_absolute_percentage_error(ata_train, ata_train_pred))
#
# print("ETA test MAPE ", mean_absolute_percentage_error(ata_test, eta_edd_h3_test_transformed[:, 15]))
# print("prediction test MAPE", mean_absolute_percentage_error(ata_test, ata_test_pred))

ETA train MAE 278.5157573703829
prediction train MAE 234.0541220760237
ETA test MAE  284.29421221864953
prediction test MAE 234.43059718522406
ETA train MSE 246574.0464249407
prediction train MSE 136276.85342773492
ETA test MSE  252857.3199356913
prediction test MSE 129113.47477425056
ETA train explained_variance_score 0.7758751853508523
prediction train explained_variance_score 0.8438333836678524
ETA test explained_variance_score  0.7764103810242154
prediction test explained_variance_score 0.8549435481180574
ETA train median_absolute_error 122.5
prediction train median_absolute_error 147.08303153043994
ETA test median_absolute_error  124.0
prediction test median_absolute_error 150.57483889916506
ETA train r2_score 0.7174381889736823
prediction train r2_score 0.8438333836678525
ETA test r2_score  0.7158508788156106
prediction test r2_score 0.8549083712526228
ETA train MAPE 0.21066943334957403
prediction train MAPE 0.2367096356321409
ETA test MAPE  0.21635844837406298
prediction test MA

In [9]:
rides_p4 = rides[rides['eta'] > 1800]
rides_p4

,eta,ata,a_t_a_result.ride_a_t_a,source_lat,source_lng,destination_lat,destination_lng,e_d_d,source_h3_4,dest_h3_4
2,1826,2075,2060,35.677704,51.332360,35.785572,51.339230,13.912000,842cf31ffffffff,842cf31ffffffff
7,2085,1874,1879,35.780780,51.329643,35.723328,51.423737,17.827000,842cf31ffffffff,842cf31ffffffff
11,1846,2102,2500,35.713715,51.315964,35.805560,51.411820,20.403000,842cf31ffffffff,842cf31ffffffff
18,2913,4069,4036,35.564354,51.237858,35.669804,51.036410,37.376999,842cf35ffffffff,842cf3dffffffff
20,1969,2271,2258,35.709396,51.389660,35.789024,51.338500,16.684000,842cf31ffffffff,842cf31ffffffff
...,...,...,...,...,...,...,...,...,...,...
20772,2134,4072,4337,35.777378,51.447437,35.732320,51.333805,17.971001,842cf31ffffffff,842cf31ffffffff
20781,1891,3041,3008,35.710052,51.389904,35.675682,51.483303,12.630000,842cf31ffffffff,842cf31ffffffff
20788,3148,3542,3460,35.670616,51.357600,35.725754,51.582650,41.417000,842cf31ffffffff,842cf31ffffffff
20796,2694,1947,1969,35.704956,51.229440,35.818474,50.953747,35.577999,842cf31ffffffff,842cf3dffffffff


In [21]:
p4_eta_edd_h3 = rides_p4[["eta", "source_h3_4", "dest_h3_4"]]
p4_ata = rides_p4["a_t_a_result.ride_a_t_a"]

In [22]:
p4_eta_edd_h3_train, p4_eta_edd_h3_test, p4_ata_train, p4_ata_test = train_test_split(p4_eta_edd_h3, p4_ata, test_size=0.05, random_state=73)
print(len(p4_eta_edd_h3_train))
print(len(p4_eta_edd_h3_test))

1860
98


In [23]:
p4_dummy_train_rides = pd.get_dummies(p4_eta_edd_h3_train, prefix={'source_h3_4': 'source_h3_4',
                                                             'dest_h3_4': 'dest_h3_4'},
                                   drop_first=True)
p4_dummy_train_rides.describe()

,eta,source_h3_4_842cf31ffffffff,source_h3_4_842cf33ffffffff,source_h3_4_842cf35ffffffff,source_h3_4_842cf37ffffffff,source_h3_4_842cf3dffffffff,dest_h3_4_842cd59ffffffff,dest_h3_4_842cf01ffffffff,dest_h3_4_842cf07ffffffff,dest_h3_4_842cf27ffffffff,dest_h3_4_842cf31ffffffff,dest_h3_4_842cf33ffffffff,dest_h3_4_842cf35ffffffff,dest_h3_4_842cf37ffffffff,dest_h3_4_842cf39ffffffff,dest_h3_4_842cf3dffffffff
count,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000,1860.000000
mean,2369.126882,0.844086,0.016129,0.061290,0.019355,0.058065,0.000538,0.000538,0.001613,0.000538,0.787634,0.016667,0.035484,0.012903,0.022581,0.120968
std,541.964843,0.362871,0.126006,0.239927,0.137806,0.233928,0.023187,0.023187,0.040139,0.023187,0.409092,0.128054,0.185049,0.112887,0.148602,0.326177
min,1801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1979.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2210.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2588.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6541.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
p4_geo_noise = np.count_nonzero(p4_dummy_train_rides, axis=0) < 10

p4_cols = p4_dummy_train_rides.columns
for col, noise in zip(p4_cols, p4_geo_noise):
    if noise:
        # DOESN'T REINDEX
        p4_eta_edd_h3_train = p4_eta_edd_h3_train[p4_dummy_train_rides[col] == 0]
        p4_ata_train = p4_ata_train[p4_dummy_train_rides[col] == 0]

p4_eta_edd_h3_train.describe()

/var/folders/yq/707ysz_j3jlggmtk5clx13rm0000gn/T/ipykernel_54626/2647439264.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p4_eta_edd_h3_train = p4_eta_edd_h3_train[p4_dummy_train_rides[col] == 0]


,eta
count,1854.000000
mean,2361.861920
std,525.054525
min,1801.000000
25%,1978.250000
50%,2208.000000
75%,2585.750000
max,5231.000000


In [25]:
print(np.count_nonzero(p4_dummy_train_rides, axis=0))
print(p4_dummy_train_rides.columns)

[1860 1570   30  114   36  108    1    1    3    1 1465   31   66   24
   42  225]
Index(['eta', 'source_h3_4_842cf31ffffffff', 'source_h3_4_842cf33ffffffff',
       'source_h3_4_842cf35ffffffff', 'source_h3_4_842cf37ffffffff',
       'source_h3_4_842cf3dffffffff', 'dest_h3_4_842cd59ffffffff',
       'dest_h3_4_842cf01ffffffff', 'dest_h3_4_842cf07ffffffff',
       'dest_h3_4_842cf27ffffffff', 'dest_h3_4_842cf31ffffffff',
       'dest_h3_4_842cf33ffffffff', 'dest_h3_4_842cf35ffffffff',
       'dest_h3_4_842cf37ffffffff', 'dest_h3_4_842cf39ffffffff',
       'dest_h3_4_842cf3dffffffff'],
      dtype='object')


In [26]:
p4_ohe = ColumnTransformer(
    [('OHE', OneHotEncoder(handle_unknown='ignore', sparse=False), ['source_h3_4', 'dest_h3_4'])],
    remainder='passthrough'
).fit(p4_eta_edd_h3_train)

In [27]:
p4_eta_edd_h3_train_transformed = p4_ohe.transform(p4_eta_edd_h3_train)
p4_eta_edd_h3_test_transformed = p4_ohe.transform(p4_eta_edd_h3_test)

In [28]:
p4_reg = LinearRegression().fit(p4_eta_edd_h3_train_transformed, p4_ata_train)
print(p4_reg.coef_)
print(p4_reg.intercept_)

[ 600.86377725  195.68473626  -47.7876749  -332.85178434 -199.32540932
 -216.58364495  368.01465972  202.72331235 -247.29678868  -88.98770215
  -40.35822013  -85.39666873 -108.69859238    1.45952628]
-629.6304105979962


In [29]:
p4_ata_train_pred = p4_reg.predict(p4_eta_edd_h3_train_transformed)
p4_ata_test_pred = p4_reg.predict(p4_eta_edd_h3_test_transformed)

In [30]:
p4_eta_edd_h3_train_transformed

array([[0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.814e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        2.764e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        2.491e+03],
       ...,
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        2.561e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.820e+03],
       [0.000e+00, 1.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        2.180e+03]])

In [31]:
print("ETA train MAE", mean_absolute_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MAE", mean_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAE ", mean_absolute_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MAE", mean_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train MSE", mean_squared_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MSE", mean_squared_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MSE ", mean_squared_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MSE", mean_squared_error(p4_ata_test, p4_ata_test_pred))

print("ETA train explained_variance_score", explained_variance_score(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train explained_variance_score", explained_variance_score(p4_ata_train, p4_ata_train_pred))

print("ETA test explained_variance_score ", explained_variance_score(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test explained_variance_score", explained_variance_score(p4_ata_test, p4_ata_test_pred))

print("ETA train median_absolute_error", median_absolute_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train median_absolute_error", median_absolute_error(p4_ata_train, p4_ata_train_pred))

print("ETA test median_absolute_error ", median_absolute_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test median_absolute_error", median_absolute_error(p4_ata_test, p4_ata_test_pred))

print("ETA train r2_score", r2_score(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train r2_score", r2_score(p4_ata_train, p4_ata_train_pred))

print("ETA test r2_score ", r2_score(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test r2_score", r2_score(p4_ata_test, p4_ata_test_pred))

print("ETA train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_eta_edd_h3_train_transformed[:, 13]))
print("prediction train MAPE", mean_absolute_percentage_error(p4_ata_train, p4_ata_train_pred))

print("ETA test MAPE ", mean_absolute_percentage_error(p4_ata_test, p4_eta_edd_h3_test_transformed[:, 13]))
print("prediction test MAPE", mean_absolute_percentage_error(p4_ata_test, p4_ata_test_pred))

ETA train MAE 800.4816612729234
prediction train MAE 558.7984343258079
ETA test MAE  819.6224489795918
prediction test MAE 629.4560778036163
ETA train MSE 1130700.4795037757
prediction train MSE 541856.5510142327
ETA test MSE  1193228.663265306
prediction test MSE 619514.2438497327
ETA train explained_variance_score 0.4215992034822401
prediction train explained_variance_score 0.48614946586327235
ETA test explained_variance_score  0.5047123312921313
prediction test explained_variance_score 0.5302767059262432
ETA train median_absolute_error 613.5
prediction train median_absolute_error 439.16264793568416
ETA test median_absolute_error  561.0
prediction test median_absolute_error 513.8574018037546
ETA train r2_score -0.07225988917942971
prediction train r2_score 0.48614946586327235
ETA test r2_score  0.09302577917044452
prediction test r2_score 0.5291066449319013
ETA train MAPE 0.23463506170168916
prediction train MAPE 0.199524369758749
ETA test MAPE  0.2363923695013324
prediction test MAP